In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# import W-Team specific packages
import sys
sys.path.append('../')
import nuftio

In [39]:
# Working directory stuff
import sys, os
PATH = '/Users/bane/Documents/school/Masters/RESEARCH/LLNL/Kimberlina1.2/prod07-sim0100-sim00199/'

SIM_DIR = r'sim01{:02}'
MESH_NAME = r'sim01{:02}.mesh_k16.prod07.trans.genmsh'
TMESH_NAME = r'sim01{:02}_mesh.json'
TARGET_DIR = 'W31-0.2'
ROCKTAB_NAME = r'sim01{:02}.usnt.rocktab'



def getSimDirName(sim):
     return os.path.join(PATH, SIM_DIR.format(sim))

def getMeshName(sim):
    return os.path.join(getSimDirName(sim), MESH_NAME.format(sim))

def getTMeshName(sim):
    return os.path.join(getSimDirName(sim), TMESH_NAME.format(sim))

def getRockTabName(sim):
    return os.path.join(os.path.join(getSimDirName(sim), TARGET_DIR), ROCKTAB_NAME.format(sim))

def getLookupTableName(sim):
    return os.path.join(getSimDirName(sim), 'lookup_table.csv')

def getLithDefName(sim):
    return os.path.join(getSimDirName(sim), 'lithologies.txt')

def getTargetDirName(sim):
    return os.path.join(getSimDirName(sim), TARGET_DIR)

def getLithMapName(sim):
    return os.path.join(getTargetDirName(sim), 'rocktab_mapper.csv')

In [42]:
# for each directory
def routine(sim):
    print('On simulation directory: {}'.format(getSimDirName(sim)))
    # find/get the genmsh file
    fname_mesh = getMeshName(sim)
    # find/get the rocktab file name
    fname_rtab = getRockTabName(sim)
    # read/pars the usnt into a TensorMesh
    usnt = nuftio.read_usnt(fname_mesh, fname_rtab)
    mesh = usnt.toTensorMesh()
    litho = usnt.definitions
    # save defintions array, lookup table, and serialized tensor mesh in that directory
    mesh.save(getTMeshName(sim))
    usnt.lookup_table.to_csv(getLookupTableName(sim), index=False)
    np.savetxt(getLithDefName(sim), usnt.definitions, fmt='%d', header='{} Lithologies'.format(SIM_DIR.format(sim)), comments='! ')
    # convert the rock tab file to a lithology definitons file
    usnt.saveLithLookupTable(getLithMapName(sim))
    # Optional - convert ntab grids to model dictionarys of flow through time
    return True

In [ ]:
# TODO: parallelize this
# for sim in range(100):
#     routine(sim)